In [1]:
import os

os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

working dir /home/tales/dev/master/mdc_analysis


In [32]:
import pandas as pd
import threading

from src.dao import objects_dao, csv_dao
from src.data_processment.input_data_version import InputDataManager

In [3]:
def len_multi_trip(multi_trip):
    soma_categs_multi_trip = 0
    for trip in multi_trip:
        soma_categs_multi_trip += len(trip)
    return soma_categs_multi_trip

In [46]:
# def load_stop_region_sequence_multi_thread(n_threads):
#     user_ids = list(multi_trip_dm.keys())[0:5]
    
#     length = int(len(user_ids) / n_threads)
#     missing = len(user_ids) % n_threads
    
#     print(user_ids)
#     print()
    
#     for i in range(n_threads):
#         init_i = i * length
#         end_i = (i+1) * length
# #         print(user_ids[init_i : end_i])
# #         print(len(user_ids[init_i : end_i]))
#         t = threading.Thread(target=load_stop_region_sequence,
#                              args=(user_ids[init_i : end_i],))
#         t.start()

#     for j in range(missing):
#         init_i = end_i
#         end_i = end_i + missing
# #         print(user_ids[init_i: end_i])
# #         print(len(user_ids[init_i : end_i]))
#         t = threading.Thread(target=load_stop_region_sequence,
#                              args=(user_ids[init_i : end_i],))
#         t.start()
    
 
# aaa = load_stop_region_sequence_multi_thread(2)
    
# print(aaa)

# def load_stop_region_sequence(user_ids):
#     srs = []
#     print("to_load:", user_ids)
#     for user_id in user_ids:
#         print("loading:", user_id)
#         srs.append(csv_dao.stop_region_sequence(user_id))
#         print("loaded", user_id)
        
#     print(srs)
#     return srs

In [25]:
data_manager = InputDataManager()

Loading Users Sequence Report


In [5]:
raw_data_dm = data_manager.get_input_data("raw_tags-0.0")["user_data"]

In [6]:
seq_report = objects_dao.load_users_sequence_report(use_cache=True)

In [7]:
multi_trip_dm = data_manager.users_multi_trip(gap_tresh_minutes=60 * 12,
                               sr_stay_time_minutes=4,
                               version="0.0.categ_v1")

In [8]:
centroids = csv_dao.load_all_users_stop_regions_centroids()

All Stop Regions:    120607


In [ ]:
srs = []
for user_id in list(multi_trip_dm.keys())[0:3]:
    print(user_id)
    srs.append(csv_dao.stop_region_sequence(user_id))

6015


In [48]:
srs

[]

### Multi Trip Has repeated first element

In [10]:
firsts_sr = []

for user_id in multi_trip_dm.keys():
    firsts_sr.append({"user_id": user_id, 
                      "len": len_multi_trip(multi_trip_dm[user_id]),
                      "1st": multi_trip_dm[user_id][0][0][0],
                      "2nd": multi_trip_dm[user_id][0][1][0]})

pd.DataFrame(firsts_sr)[["user_id", "len", "1st", "2nd"]].head()

,user_id,len,1st,2nd
0,6015,184,liquor_store,liquor_store
1,6086,290,HOME,HOME
2,6014,275,WORK,WORK
3,6057,357,store,store
4,6181,20,laundry,laundry


### Different Lengths fo Stop Regions

In [ ]:
lengths = []

for user_id in multi_trip_dm.keys():
    
    lengths.append({"user_id": user_id,
                   "raw_data_dm": len(raw_data_dm[user_id]),
                   "multi_trip_dm": len(multi_trip_dm[user_id]), 
                   "seq_report": len(seq_report[user_id]), 
                   "srs": len(srs[user_id]),
                   "centroids": len(centroids[centroids["user_id"] == int(user_id)])})
    
pd.DataFrame(lengths)

In [ ]:
from itertools import groupby

len([x[0] for x in groupby(centroids["user_id"].tolist())])